In [64]:
import glob
import os
import subprocess

import cdpybio as cpb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import ciepy as cpy
import projectpy as ppy

%matplotlib inline

In [65]:
array_meta = pd.read_table(os.path.join(cpy.root, 'data', 'array_metadata.tsv'), index_col=0)
wgs_meta = pd.read_table(os.path.join(cpy.root, 'data', 'wgs_metadata.tsv'), index_col=0, 
                         squeeze=True)
fn = ('/raid3/projects/CARDIPS/analysis/RNAseq/150512_150522_150527_metadata.tsv')
rna_meta = pd.read_table(fn, index_col=0)

# Metadata

In [66]:
rna_meta = rna_meta[rna_meta.censor == False]
rna_meta = rna_meta[rna_meta.cell_type == 'iPSC']

rna_meta['wgs_id'] = np.nan
for i in rna_meta.index:
    tdf = wgs_meta[wgs_meta == rna_meta.ix[i, 'subject']]
    if tdf.shape[0] == 1:
        rna_meta.ix[i, 'wgs_id'] = tdf.index[0]
rna_meta = rna_meta.dropna(subset=['wgs_id'])
rna_meta.to_csv(os.path.join(cpy.root, 'data', 'rna_seq_metadata.tsv'), sep='\t')

# Gene Counts

In [33]:
fns = glob.glob(os.path.join('/raid3/projects/CARDIPS/analysis/RNAseq/*XX/cdeboever/'
                             'results/*_counts/gene_counts.tsv'))
count_fns = []
for fn in fns:
    s = fn.split(os.path.sep)[-2].split('_')[0]
    if s in rna_meta.index:
        count_fns.append(fn)

In [55]:
counts = [pd.read_table(fn, index_col=0, header=None, squeeze=True) for fn in count_fns]
sample_names = [fn.split(os.path.sep)[-2].split('_')[0] for fn in count_fns]
d = {key: value for (key, value) in zip(sample_names, counts)}
counts = pd.DataFrame(d)
counts.index.name = 'gene'
counts.to_csv(os.path.join(cpy.root, 'data', 'gene_counts.tsv'), sep='\t')